# Setting variables



In [1]:
import pandas as pd
import warnings
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import os
from lightgbm import LGBMRegressor
from collections import deque



ais_train = pd.read_csv('/Users/danialbashir/Desktop/Projects/TDT4173-Main-Project-1/Data/ais_train.csv', delimiter='|')
ais_test= pd.read_csv('/Users/danialbashir/Desktop/Projects/TDT4173-Main-Project-1/Data/ais_test.csv')
vessels = pd.read_csv('/Users/danialbashir/Desktop/Projects/TDT4173-Main-Project-1/Data/vessels.csv', delimiter='|')
ports = pd.read_csv('/Users/danialbashir/Desktop/Projects/TDT4173-Main-Project-1/Data/ports.csv', delimiter='|')

In [2]:
# =========================
# Parameters
# =========================

number_of_days = 5  # 0 if not just predicting
remove_fraction = 0
max_depth = 23
n_estimators = 50
mode = 'predict'  # or 'validate'
remove_anomalies_speed = True  # Set to True to remove anomalies, False to keep all records
MAX_REALISTIC_SPEED_KNOTS = 30  # Speed threshold for anomalies
trim_train_data = True
remove_cog_sog_anomalies=True
PORT_PROXIMITY_THRESHOLD= 50000 # Distance threshold for port proximity

number_of_lagged_coordinates = 1
list_of_features = ['lat_diff', 'lon_diff']
moving_avg_features = []
# New parameter for rolling window size
rolling_window_size = 20  # You can adjust this as needed


# Split data

In [3]:
import pandas as pd

ais_test['time'] = pd.to_datetime(ais_test['time'])
ais_train['time'] = pd.to_datetime(ais_train['time'])

print('Splitting data...')
if mode == 'predict':
    # Extract the target variables for the training set
    X_val = ais_test.copy()
    X_train = ais_train.copy()
    #remove all vessels that are not in the neighboorhood of the test set 
    
else:
    # Sort the entire dataset by 'time' to ensure order
    ais_sorted = ais_train.sort_values(by='time')

    # Calculate the threshold date for the last number_of_days days
    threshold_date = ais_sorted['time'].max() - pd.Timedelta(days=number_of_days)

    # Identify validation set indices where 'time' is within the last number_pf_days_days days
    val_indices = ais_sorted[ais_sorted['time'] > threshold_date].index

    # Use these indices to create the validation set
    X_val = ais_train.loc[val_indices].reset_index(drop=True)

    # Drop these indices from ais_train to get the training set
    X_train = ais_train.drop(val_indices).reset_index(drop=True)

    # Calculate the minimum date in the validation set (i.e., the first validation day)
    first_val_time = X_val['time'].min()

    # Add a column to the validation set for days since the first validation date
    X_val['days_since_last_train'] = (pd.to_datetime(X_val['time']) - pd.to_datetime(first_val_time)).dt.days + 1

    # Cap the days_since_last_train at 5
    X_val['days_since_last_train'] = X_val['days_since_last_train'].apply(lambda day: min(day, 5))

    # Remove a random x% of rows for each day in the validation set
    if 0 < remove_fraction < 1:
        # Group by 'days_since_last_train' and sample a fraction of rows from each day
        X_val = X_val.groupby('days_since_last_train', group_keys=False).apply(
            lambda group: group.sample(frac=(1 - remove_fraction), random_state=42)
        ).reset_index(drop=True)
    
print('Data split done')


Splitting data...
Data split done


# Feature engeneering

In [4]:
# Copy datasets
ais_test = X_val.copy()
ais_train = X_train.copy()

# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Ensure 'time' is in datetime format and sort data
ais_train['time'] = pd.to_datetime(ais_train['time'])
ais_train.sort_values(['vesselId', 'time'], inplace=True)

# Convert relevant columns to numeric types
numeric_columns = ['latitude_vessel', 'longitude_vessel', 'cog', 'sog']
ais_train[numeric_columns] = ais_train[numeric_columns].apply(pd.to_numeric, errors='coerce')

print(f"Initial training records: {len(ais_train)}")

# =========================
# Anomaly Detection and Interpolation for Latitude/Longitude
# =========================
print(f"Interpolating records with speed_knots > {MAX_REALISTIC_SPEED_KNOTS}...")

# Calculate shifted coordinates
ais_train['prev_latitude'] = ais_train.groupby('vesselId')['latitude_vessel'].shift(1)
ais_train['prev_longitude'] = ais_train.groupby('vesselId')['longitude_vessel'].shift(1)

# Function to compute distance between two points
def compute_distance(row):
    if pd.isnull(row['prev_latitude']) or pd.isnull(row['prev_longitude']):
        return np.nan
    else:
        point1 = (row['prev_latitude'], row['prev_longitude'])
        point2 = (row['latitude_vessel'], row['longitude_vessel'])
        return geodesic(point1, point2).meters

# Calculate distance in meters between consecutive points
ais_train['distance_meters'] = ais_train.apply(compute_distance, axis=1)

# Calculate time difference in seconds within each 'vesselId'
ais_train['time_diff_seconds'] = ais_train.groupby('vesselId')['time'].diff().dt.total_seconds()
ais_train['time_diff_seconds'].replace(0, np.nan, inplace=True)  # Avoid division by zero

# Calculate speed in meters per second and convert to knots
ais_train['speed_m_s'] = ais_train['distance_meters'] / ais_train['time_diff_seconds']
ais_train['speed_knots'] = ais_train['speed_m_s'] * 1.94384

# Flag speed anomalies
ais_train['speed_anomaly'] = ais_train['speed_knots'] > MAX_REALISTIC_SPEED_KNOTS

# Set anomalies to NaN in 'latitude_vessel' and 'longitude_vessel'
ais_train.loc[ais_train['speed_anomaly'], ['latitude_vessel', 'longitude_vessel']] = np.nan

# =========================
# Anomaly Detection and Interpolation for COG/SOG
# =========================
print("Interpolating out-of-range COG and SOG values...")

# Flag COG anomalies
ais_train['cog_anomaly'] = (ais_train['cog'] >= 360) | (ais_train['cog'] < 0)
ais_train['sog_anomaly'] = ais_train['sog'] > 30

# Set out-of-range COG and SOG values to NaN for interpolation
ais_train.loc[ais_train['cog_anomaly'], 'cog'] = np.nan
ais_train.loc[ais_train['sog_anomaly'], 'sog'] = np.nan

# =========================
# Interpolation for All Features
# =========================
# Set 'time' as the index for interpolation
ais_train.set_index('time', inplace=True)

# Interpolate within each 'vesselId' group and fill missing values forward/backward
ais_train[['latitude_vessel', 'longitude_vessel', 'cog', 'sog']] = ais_train.groupby('vesselId')[
    ['latitude_vessel', 'longitude_vessel', 'cog', 'sog']
].transform(lambda group: group.interpolate(method='time').ffill().bfill())

# Reset the index after interpolation
ais_train.reset_index(inplace=True)

# Drop the anomaly flag columns as they're no longer needed
ais_train.drop(columns=['speed_anomaly', 'cog_anomaly', 'sog_anomaly'], inplace=True)

# Drop any remaining NaN values resulting from shifts or interpolation limits
ais_train.dropna(subset=['latitude_vessel', 'longitude_vessel', 'cog', 'sog'], inplace=True)

print(f"Anomalies interpolated. Remaining training records: {len(ais_train)}")


Initial training records: 1522065
Interpolating records with speed_knots > 30...
Interpolating out-of-range COG and SOG values...
Anomalies interpolated. Remaining training records: 1522065


In [5]:
print(len(ais_train))
#remove all entries that have cog>=360 or cog<0
ais_train = ais_train[ais_train['cog'] < 360]
ais_train = ais_train[ais_train['cog'] >= 0]
print(len(ais_train))


1522065
1522065


In [6]:
#merge the datasets with vessels
ais_train = pd.merge(ais_train, vessels, on='vesselId', how='left')
ais_test = pd.merge(ais_test, vessels, on='vesselId', how='left')

#merge ais_train with ports
ais_train = pd.merge(ais_train, ports, on='portId', how='left', suffixes=('', '_port'))


# Ensure the DataFrames are sorted by 'vesselId' and 'time'
ais_train = ais_train.sort_values(['vesselId', 'time'])
ais_test = ais_test.sort_values(['vesselId', 'time'])

# 3. Calculate the time difference between consecutive rows within each 'vesselId'
ais_train['time_diff_seconds'] = ais_train.groupby('vesselId')['time'].diff().dt.total_seconds()


# 4. Create lagged time difference features
for i in range(1, number_of_lagged_coordinates + 2):
    if i == 1:
        # For lag 1, use the current 'time_diff_seconds' without shifting
        ais_train[f'prev{i}_time_diff_seconds'] = ais_train['time_diff_seconds']
        ais_test[f'prev{i}_time_diff_seconds'] = np.nan
    else:
        # For lags >1, shift 'time_diff_seconds' by (i -1)
        ais_train[f'prev{i}_time_diff_seconds'] = ais_train.groupby('vesselId')['time_diff_seconds'].shift(i - 1)
        ais_test[f'prev{i}_time_diff_seconds'] = np.nan
    
    # Append the feature name to the list of features
for i in range(1, number_of_lagged_coordinates + 1):
    list_of_features.append(f'prev{i}_time_diff_seconds')


# 5. (Optional) Drop the intermediate 'time_diff_seconds' column if not needed
ais_train.drop(columns=['time_diff_seconds'], inplace=True)


#generate number_of_lagged_coordinates lagged coordinates and corresponting time differences
for i in range(1, number_of_lagged_coordinates + 3):
    ais_train[f'prev{i}_longitude'] = ais_train.groupby('vesselId')['longitude_vessel'].shift(i)
    ais_train[f'prev{i}_latitude'] = ais_train.groupby('vesselId')['latitude_vessel'].shift(i)
    ais_test[f'prev{i}_longitude'] = np.nan
    ais_test[f'prev{i}_latitude'] = np.nan
for i in range (1, number_of_lagged_coordinates + 1):
    list_of_features.extend([f'prev{i}_longitude', f'prev{i}_latitude'])
    
        
if moving_avg_features:
    for feature in moving_avg_features.copy():
        if feature=='cog':
            
            ais_train['cog_rad'] = np.radians(ais_train['cog'])
            
            ais_train['cog_sin'] = np.sin(ais_train['cog_rad'])
            ais_train['cog_cos'] = np.cos(ais_train['cog_rad'])
            
            
            # Compute moving averages for sine and cosine components
            ais_train['cog_sin_moving_avg'] = ais_train.groupby('vesselId')['cog_sin'].transform(
                lambda x: x.rolling(window=rolling_window_size, min_periods=1).mean()
            )
            ais_train['cog_cos_moving_avg'] = ais_train.groupby('vesselId')['cog_cos'].transform(
                lambda x: x.rolling(window=rolling_window_size, min_periods=1).mean()
            )
            
            ais_train['cog_avg_rad'] = np.arctan2(ais_train['cog_sin_moving_avg'], ais_train['cog_cos_moving_avg'])
            ais_train['cog_avg_deg'] = (np.degrees(ais_train['cog_avg_rad'])) % 360
            
            # Update feature lists
            moving_avg_features.remove('cog')  # Remove the original 'cog'
            moving_avg_features.extend(['cog_sin', 'cog_cos'])  # Add sine and cosine moving averages
            list_of_features.extend(['cog_avg_deg'])  # Add the reconstructed averaged COG
            ais_test['cog_avg_deg'] = np.nan
            ais_test['cog_sin_moving_avg'] = np.nan
            ais_test['cog_cos_moving_avg'] = np.nan
            ais_test['cog_avg_rad'] = np.nan
               
        else:
            ais_train[f'{feature}_moving_avg'] = ais_train.groupby('vesselId')[feature].transform(lambda x: x.rolling(rolling_window_size, min_periods=1).mean())
            ais_test[f'{feature}_moving_avg'] = np.nan
            list_of_features.append(f'{feature}_moving_avg')

#drop rows with NaN values due to shift
ais_train.dropna(subset=['prev1_longitude', 'prev1_latitude', 'prev2_longitude', 'prev2_latitude'], inplace=True)

print(ais_train.columns)
            
for feature in list_of_features.copy():
    if feature == 'weekday':      
        ais_train['weekday'] = ais_train['time'].dt.weekday.astype(int)
        ais_test['weekday'] = ais_test['time'].dt.weekday.astype(int)
    elif feature == 'month':
        ais_train['month'] = ais_train['time'].dt.month
        ais_test['month'] = ais_test['time'].dt.month
    elif feature == 'hour':
        ais_train['hour'] = ais_train['time'].dt.hour
        ais_test['hour'] = ais_test['time'].dt.hour
    # seconds after 1970
    elif feature == 'total_seconds':
        # Calculate `total_seconds` in a consistent manner across training and test sets
        reference_time = min(ais_train['time'].min(), ais_test['time'].min())
        # Update `total_seconds` calculation
        ais_train['total_seconds'] = (ais_train['time'] - reference_time).dt.total_seconds()
        ais_test['total_seconds'] = (ais_test['time'] - reference_time).dt.total_seconds()
    elif feature == 'day_of_year':
        ais_train['day_of_year'] = ais_train['time'].dt.dayofyear
        ais_test['day_of_year'] = ais_test['time'].dt.dayofyear
    elif feature == 'lon_diff':
        ais_train['lon_diff'] = ((ais_train['prev1_longitude'] - ais_train['prev2_longitude'])/ais_train['prev2_time_diff_seconds']*ais_train['prev1_time_diff_seconds'])
        ais_test['lon_diff'] = np.nan
    elif feature == 'lat_diff':
        ais_train['lat_diff'] = (((ais_train['prev1_latitude'] - ais_train['prev2_latitude'])/ais_train['prev2_time_diff_seconds'])*ais_train['prev1_time_diff_seconds'])
        ais_test['lat_diff'] = np.nan
    elif feature == 'lon_diff2':
        ais_train['lon_diff2'] = ((ais_train['prev2_longitude'] - ais_train['prev3_longitude'])/ais_train['prev2_time_diff_seconds']*(ais_train['prev1_time_diff_seconds']+ais_train['prev2_time_diff_seconds']))
        ais_test['lon_diff2'] = np.nan
    elif feature == 'lat_diff2':
        ais_train['lat_diff2'] = ((ais_train['prev2_latitude'] - ais_train['prev3_latitude'])/ais_train['prev2_time_diff_seconds']*(ais_train['prev1_time_diff_seconds']+ais_train['prev2_time_diff_seconds']))
        ais_test['lat_diff2'] = np.nan
    
    elif feature == 'distance':
        ais_train['distance'] = ais_train.apply(
        lambda x: geodesic(
            (x['prev2_latitude'], x['prev2_longitude']), 
            (x['prev1_latitude'], x['prev1_longitude'])  # Corrected
            ).meters, 
            axis=1)
    elif feature == 'xcogxtime':
        ais_train['xcogxtime'] = ais_train['xcog']*ais_train['prev1_time_diff_seconds']
        ais_test['xcogxtime'] = np.nan
    elif feature == 'ycogxtime':
        ais_train['ycogxtime'] = ais_train['ycog']*ais_train['prev1_time_diff_seconds']
        ais_test['ycogytime'] = np.nan
    elif feature=='sogxtime':
        ais_train['sogxtime'] = ais_train['sog']*ais_train['prev1_time_diff_seconds']
        ais_test['sogxtime'] = np.nan
    elif feature.startswith('lag') and ('lat_diff' in feature or 'lon_diff' in feature):
        lag_num = int(feature.split('_')[0][3:])
        diff_type = 'lat_diff' if 'lat_diff' in feature else 'lon_diff'
        ais_train[feature] = ais_train.groupby('vesselId')[diff_type].shift(lag_num)
        ais_test[feature] = ais_test.groupby('vesselId')[diff_type].shift(lag_num)
    elif feature == 'port_latitude':
        ais_train['port_latitude'] = ais_train['latitude']
        ais_test['port_latitude'] = np.nan
    elif feature == 'port_longitude':
        ais_train['port_longitude'] = ais_train['longitude']
        ais_test['port_longitude'] = np.nan
    elif feature == 'length':
        ais_train['length'] = ais_train['length'].astype(int)
        ais_test['length'] = ais_test['length'].astype(int)
    
if 'port_latitude' in list_of_features and 'port_longitude' in list_of_features:
    # Calculate the distance to the port
    ais_train['distance_to_port'] = ais_train.apply(
        lambda x: np.nan if pd.isna(x['port_latitude']) else geodesic(
            (x['prev1_latitude'], x['prev1_longitude']),
            (x['port_latitude'], x['port_longitude'])
        ).meters,
        axis=1
    )

    # Set 'port_latitude' and 'port_longitude' to NaN if 'distance_to_port' is NaN
    ais_train.loc[ais_train['distance_to_port'].isna(), ['port_latitude', 'port_longitude']] = -12345

    # Set 'port_latitude' and 'port_longitude' to NaN if 'distance_to_port' exceeds the threshold
    ais_train.loc[ais_train['distance_to_port'] > PORT_PROXIMITY_THRESHOLD, ['port_latitude', 'port_longitude']] = -12345


#drop rows with NaN values due to shift
ais_train.dropna(subset=list_of_features, inplace=True)
        

#sort by vesselId and time
ais_train.sort_values(by=['vesselId', 'time'], inplace=True)
ais_test.sort_values(by=['vesselId', 'time'], inplace=True)



Index(['time', 'cog', 'sog', 'rot', 'heading', 'navstat', 'etaRaw',
       'latitude_vessel', 'longitude_vessel', 'vesselId', 'portId',
       'prev_latitude', 'prev_longitude', 'distance_meters', 'speed_m_s',
       'speed_knots', 'shippingLineId', 'CEU', 'DWT', 'GT', 'NT', 'vesselType',
       'breadth', 'depth', 'draft', 'enginePower', 'freshWater', 'fuel',
       'homePort', 'length', 'maxHeight', 'maxSpeed', 'maxWidth',
       'rampCapacity', 'yearBuilt', 'name', 'portLocation', 'longitude',
       'latitude', 'UN_LOCODE', 'countryName', 'ISO',
       'prev1_time_diff_seconds', 'prev2_time_diff_seconds', 'prev1_longitude',
       'prev1_latitude', 'prev2_longitude', 'prev2_latitude',
       'prev3_longitude', 'prev3_latitude'],
      dtype='object')


# Trim train set

In [7]:

if trim_train_data:
    # Find the median number of rows per vessel in the training set
    median_rows_per_vessel = ais_train.groupby('vesselId').size().median()
    print(median_rows_per_vessel)

    # Calculate the number of rows per vessel in the training set
    rows_per_vessel = ais_train.groupby('vesselId').size()

    # Find vessels with fewer than the median number of rows
    vessels_with_few_rows = set(rows_per_vessel[rows_per_vessel < 100].index)

    # Find vessels that are in the training set but not in the test set
    vessels_not_in_test = set(ais_train['vesselId']) - set(ais_test['vesselId'])

    # Combine conditions: vessels with few rows AND not in test set
    vessels_to_remove = vessels_with_few_rows & vessels_not_in_test

    # Remove rows corresponding to these vessels from the training set
    ais_train = ais_train[~ais_train['vesselId'].isin(vessels_to_remove)]

    #print number of negative time differences
    print(ais_train[ais_train['prev1_time_diff_seconds'] < 0].shape[0])
    
if mode == 'validate':
    #ais_test = ais_test[~ais_test['vesselId'].isin(vessels_to_remove)]
    #if boat is not in train set but in test set, remove it from test set
    ais_test = ais_test[ais_test['vesselId'].isin(ais_train['vesselId'])]
    


moving_avg_dicts = {}

for feature in moving_avg_features:
    moving_avg_dicts[f"{feature}_moving_avg"] = {}
    
    
# Initialize with training data
for vessel_id, group in ais_train.groupby('vesselId'):
    #sort by time
    group.sort_values(by='time', inplace=True) 
    for feature in moving_avg_features:
        moving_avg_dicts[f"{feature}_moving_avg"][vessel_id] = deque(group[f'{feature}_moving_avg'].tail(rolling_window_size), maxlen=rolling_window_size)
        
moving_avg_dicts_pure = moving_avg_dicts.copy()
        
def update_moving_averages(vessel_id, feature, new_value):
    moving_avg_dicts[f"{feature}_moving_avg"][vessel_id].append(new_value)
    return np.mean(moving_avg_dicts[f"{feature}_moving_avg"][vessel_id])   
    

1708.0
0


# Define training features

In [8]:
#sort and group by vesselId
ais_train.sort_values(by=['vesselId', 'time'], inplace=True)

X_for_training = ais_train[list_of_features]

# Extract the target variables for the training set
y_train_long = ais_train['longitude_vessel']
y_train_lat = ais_train['latitude_vessel']

X_for_training.head()

,lat_diff,lon_diff,prev1_time_diff_seconds,prev1_longitude,prev1_latitude
2,0.078877,-0.100401,1583.0,77.49505,7.57302
3,0.062838,-0.081995,1285.0,77.39404,7.65043
4,0.061059,-0.078479,1259.0,77.31394,7.71275
5,0.042338,-0.055885,901.0,77.23585,7.77191
6,0.055026,-0.073090,1211.0,77.18147,7.81285


# Train model

In [9]:
print('Training models...')

# Train model for longitude
model_long = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, n_jobs=-1, random_state=42, verbose=0)
model_long.fit(X_for_training, y_train_long)

print('Longitude model trained')

importances_long = model_long.feature_importances_
for i in range(len(list_of_features)):
    print(f"{X_for_training.columns[i]}: {importances_long[i]}")

# Train model for latitude
model_lat = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, n_jobs=-1, random_state=42, verbose=0)
model_lat.fit(X_for_training, y_train_lat)

print('Latitude model trained')

print('All models trained.')

#print feature importance

importances_lat = model_lat.feature_importances_

    
for i in range(len(list_of_features)):
    print(f"{X_for_training.columns[i]}: {importances_lat[i]}")
    


Training models...
Longitude model trained
lat_diff: 0.0002100876049812494
lon_diff: 0.001469552008038237
prev1_time_diff_seconds: 0.003312593047416647
prev1_longitude: 0.9947905400141245
prev1_latitude: 0.00021722732543948446
Latitude model trained
All models trained.
lat_diff: 0.0014628890215073404
lon_diff: 0.0004925697711957313
prev1_time_diff_seconds: 0.0019438170544288313
prev1_longitude: 0.00036385319094865156
prev1_latitude: 0.9957368709619194


#  Predict



In [10]:
#Make copy of moving_avg_dict
moving_avg_dicts = moving_avg_dicts_pure.copy()

# Initialize the prediction set from ais_test
prediction_data = ais_test.copy()
prediction_data['predicted_longitude'] = np.nan
prediction_data['predicted_latitude'] = np.nan

# Set up groups for efficient lookup by vessel
train_set_sorted = ais_train.sort_values(by=['vesselId', 'time'])
ais_train_groups = train_set_sorted.groupby('vesselId')

# Initialize counters for progress tracking
total_rows = len(prediction_data)
progress_interval = max(total_rows // 100, 1)  # Calculate for 1% intervals, minimum 1
processed_rows = 0  # Initialize counter for processed rows

# Set model to use a single thread for faster individual row processing
model_long.n_jobs = 1
model_lat.n_jobs = 1

# Loop through each vessel to perform sequential predictions
for vessel_id, group in prediction_data.groupby('vesselId'):
    # Get past data for this vessel
    past_data = ais_train_groups.get_group(vessel_id)
    # Track last known position within each vessel group
    last_known_data = past_data.iloc[-1].copy()

    last_known_data['predicted_longitude'] = last_known_data['longitude_vessel']
    last_known_data['predicted_latitude'] = last_known_data['latitude_vessel']
    

    # Iterate over each row for this vessel in prediction_data
    for idx, (index, row) in enumerate(group.iterrows()):
        dict_avgs = {}
        
        
        if 'cog_avg_deg' in list_of_features:
            #calculate cog based on earlier coordinates in degrees
            cog = np.degrees(np.arctan2(
                last_known_data['predicted_longitude'] - last_known_data['prev1_longitude'], 
                last_known_data['predicted_latitude'] - last_known_data['prev1_latitude']
            )) % 360
            
            
            cog_conservative = 0.90 * cog + 0.1 * last_known_data['cog_avg_deg']
            
            # Decompose the updated COG into sine and cosine
            cog_conservative_rad = np.radians(cog_conservative)
            cog_sin_new = np.sin(cog_conservative_rad)
            cog_cos_new = np.cos(cog_conservative_rad)
            
            # Update moving averages using deque or other methods
            dict_avgs['cog_sin'] = update_moving_averages(vessel_id, 'cog_sin', cog_sin_new)
            dict_avgs['cog_cos'] = update_moving_averages(vessel_id, 'cog_cos', cog_cos_new)

            # Reconstruct the averaged COG
            cog_avg_rad = np.arctan2(dict_avgs['cog_sin'], dict_avgs['cog_cos'])
            cog_avg_deg = (np.degrees(cog_avg_rad)) % 360
            
            # Assign the averaged COG to the current prediction
            prediction_data.at[index, 'cog_avg_deg'] = cog_avg_deg
            
        #calculate sog based on earlier coordinates
            sog = (geodesic(
                (last_known_data['predicted_latitude'], last_known_data['predicted_longitude']), 
                (last_known_data['prev1_latitude'], last_known_data['prev1_longitude'])
            ).meters / last_known_data['prev1_time_diff_seconds']) / 0.51444  
            sog_conservative = 0.90 * sog + 0.1 * last_known_data['sog_moving_avg']
        #calculate xcog based on earlier coordinates
        
        """
        xcog = (last_known_data['predicted_longitude'] - last_known_data['prev1_longitude'])/last_known_data['prev1_time_diff_seconds']
        xcog_conservative = 0.5 * xcog + 0.5 * last_known_data['xcog_moving_avg']
        ycog = (last_known_data['predicted_latitude'] - last_known_data['prev1_latitude'])/last_known_data['prev1_time_diff_seconds']
        ycog_conservative = 0.5 * ycog + 0.5 * last_known_data['ycog_moving_avg']
        """
        
        
        for feature in moving_avg_features:
            
            if feature == 'latitude_vessel' :
                dict_avgs[feature] = update_moving_averages( vessel_id, feature, last_known_data['predicted_latitude'])
                
            elif feature == 'longitude_vessel':
                dict_avgs[feature] = update_moving_averages( vessel_id, feature, last_known_data['predicted_longitude'])
                
            elif feature == 'sog':
                dict_avgs[feature] = update_moving_averages( vessel_id, feature, sog_conservative)
            
            
            """
            elif feature=='xcog':    
                dict_avgs[feature] = update_moving_averages( vessel_id, feature, xcog_conservative)
                
            elif feature=='ycog':
                dict_avgs[feature] = update_moving_averages( vessel_id, feature, ycog_conservative)
            """
            
        # Assign moving averages to the current row
        for feature in moving_avg_features:
            prediction_data.at[index, f'{feature}_moving_avg'] = dict_avgs[feature]
        
        # Update lagged time difference dynamically
        time_diff = (row['time'] - last_known_data['time']).total_seconds()
        
        if 'port_latitude' in list_of_features and 'port_longitude' in list_of_features:
            #if port latitude is np.nan, set it to -12345
            if pd.isna(last_known_data['port_latitude']):
                last_known_data['port_latitude'] = -12345
                last_known_data['port_longitude'] = -12345
            
            #check if port_lat is -12345
            if last_known_data['port_latitude'] != -12345:
                #calculate distance from predicted coordinates to port
                distance_to_port = geodesic((last_known_data['predicted_latitude'], last_known_data['predicted_longitude']), (last_known_data['port_latitude'], last_known_data['port_longitude'])).meters
                if distance_to_port < PORT_PROXIMITY_THRESHOLD:
                    prediction_data.at[index, 'port_latitude'] = last_known_data['port_latitude']
                    prediction_data.at[index, 'port_longitude'] = last_known_data['port_longitude']
                else:
                    prediction_data.at[index, 'port_latitude'] = -12345
                    prediction_data.at[index, 'port_longitude'] = -12345
            else:
                prediction_data.at[index, 'port_latitude'] = -12345
                prediction_data.at[index, 'port_longitude'] = -12345
        
        prediction_data.at[index, 'prev1_time_diff_seconds'] = time_diff
        prediction_data.at[index, 'prev2_time_diff_seconds'] = last_known_data['prev1_time_diff_seconds']
        
        # Dynamically update lagged coordinates
        prediction_data.at[index, 'prev1_longitude'] = last_known_data['predicted_longitude']
        prediction_data.at[index, 'prev1_latitude'] = last_known_data['predicted_latitude']
        prediction_data.at[index, 'prev2_latitude'] = last_known_data['prev1_latitude']
        prediction_data.at[index, 'prev2_longitude'] = last_known_data['prev1_longitude']
        
        if 'lat_diff' in list_of_features:
            prediction_data.at[index, 'lat_diff'] = (((last_known_data['predicted_latitude'] - last_known_data['prev1_latitude'])/last_known_data['prev1_time_diff_seconds'])*time_diff)
            prediction_data.at[index, 'lon_diff'] = (((last_known_data['predicted_longitude'] - last_known_data['prev1_longitude'])/last_known_data['prev1_time_diff_seconds'])*time_diff)
        if 'lat_diff2' in list_of_features:
            prediction_data.at[index, 'lat_diff2'] = (((last_known_data['prev1_latitude'] - last_known_data['prev2_latitude'])/last_known_data['prev2_time_diff_seconds'])*(last_known_data['prev1_time_diff_seconds']+last_known_data['prev2_time_diff_seconds']))
            prediction_data.at[index, 'lon_diff2'] = (((last_known_data['prev1_longitude'] - last_known_data['prev2_longitude'])/last_known_data['prev2_time_diff_seconds'])*(last_known_data['prev1_time_diff_seconds']+last_known_data['prev2_time_diff_seconds']))
            
        
        if 'distance' in list_of_features:
            prediction_data.at[index, 'distance'] = geodesic((last_known_data['prev1_latitude'], last_known_data['prev1_longitude']), (last_known_data['predicted_latitude'], last_known_data['predicted_longitude'])).meters
        
        if 'xcogxtime' in list_of_features:
            prediction_data.at[index, 'xcogxtime'] = prediction_data.at[index, 'xcog_moving_avg']*time_diff
            
        if 'ycogxtime' in list_of_features:
            prediction_data.at[index, 'ycogxtime'] = prediction_data.at[index, 'ycog_moving_avg']*time_diff
        
        if 'sogxtime' in list_of_features:
            prediction_data.at[index, 'sogxtime'] = prediction_data.at[index, 'sog_moving_avg']*time_diff
        
    # Update multiple lag features
        for feature in list_of_features:
            if feature.startswith('lag') and ('lat_diff' in feature or 'lon_diff' in feature):
                lag_num = int(feature.split('_')[0][3:])
                diff_type = 'lat_diff' if 'lat_diff' in feature else 'lon_diff'
                if lag_num == 1:
                    prediction_data.at[index, feature] = last_known_data[diff_type]
                else:
                    prev_lag_feature = f'lag{lag_num - 1}_{diff_type}'
                    prediction_data.at[index, feature] = last_known_data.get(prev_lag_feature, np.nan)

        
        if number_of_lagged_coordinates > 1:
            # Update higher-order lag features based on latest predictions
            for i in range(2, number_of_lagged_coordinates + 1):  
                # Shift previous predictions back for each lag
                prediction_data.at[index, f'prev{i}_longitude'] = last_known_data.get(f'prev{i-1}_longitude', np.nan)
                prediction_data.at[index, f'prev{i}_latitude'] = last_known_data.get(f'prev{i-1}_latitude', np.nan)
                prediction_data.at[index, f'prev{i}_time_diff_seconds'] = last_known_data.get(f'prev{i-1}_time_diff_seconds', np.nan)

        
        features = prediction_data.loc[index, list_of_features].to_frame().T

                    
        # Make predictions for the current row
        pred_long = model_long.predict(features)[0]
        pred_lat = model_lat.predict(features)[0]
        
        #pred_lat, pred_long = adjust_to_sea(pred_lat, pred_long)
        
        # Store predictions
        prediction_data.at[index, 'predicted_longitude'] = pred_long
        prediction_data.at[index, 'predicted_latitude'] = pred_lat
        
        # Update last known data with current prediction
        last_known_data = prediction_data.loc[index].copy()
        last_known_data['time'] = row['time']
        
        # Handling lag_lat_diff and lag_lon_diff
        for feature in list_of_features:
            if feature.startswith('lag') and 'lat_diff' in feature:
                prediction_data.at[index, feature] = last_known_data['lat_diff']
            if feature.startswith('lag') and 'lon_diff' in feature:
                prediction_data.at[index, feature] = last_known_data['lon_diff']
        
        
        # Increment the counter for processed rows and print progress
        processed_rows += 1
        if processed_rows % progress_interval == 0:
            percent_complete = int((processed_rows / total_rows) * 100)
            print(f"Processing: {percent_complete}% complete.")
                
print("Processing: 100% complete.")


Processing: 0% complete.
Processing: 1% complete.
Processing: 2% complete.
Processing: 3% complete.
Processing: 4% complete.
Processing: 5% complete.
Processing: 6% complete.
Processing: 7% complete.
Processing: 8% complete.
Processing: 9% complete.
Processing: 10% complete.
Processing: 11% complete.
Processing: 12% complete.
Processing: 13% complete.
Processing: 14% complete.
Processing: 15% complete.
Processing: 16% complete.
Processing: 17% complete.
Processing: 18% complete.
Processing: 19% complete.
Processing: 20% complete.
Processing: 21% complete.
Processing: 22% complete.
Processing: 23% complete.
Processing: 24% complete.
Processing: 25% complete.
Processing: 26% complete.
Processing: 27% complete.
Processing: 28% complete.
Processing: 29% complete.
Processing: 30% complete.
Processing: 31% complete.
Processing: 32% complete.
Processing: 33% complete.
Processing: 34% complete.
Processing: 35% complete.
Processing: 36% complete.
Processing: 37% complete.
Processing: 38% comple

In [14]:

#outputs
if mode=='predict':
    
    prediction_data['longitude_predicted'] = prediction_data['predicted_longitude']
    prediction_data['latitude_predicted'] = prediction_data['predicted_latitude']
    detailed_predictions = prediction_data
    
    detailed_predictions.to_csv('Detailed_Predictions.csv', index=False)
    print("Detailed predictions saved to Detailed_Predictions.csv.")

    submission_predictions = prediction_data[['ID', 'longitude_predicted', 'latitude_predicted']]
    submission_predictions.to_csv('Submission.csv', index=False)
    print("Submission predictions saved to Submission.csv.")
    


Detailed predictions saved to Detailed_Predictions.csv.
Submission predictions saved to Submission.csv.
